In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from datetime import date
from plot_hydrograph import plot_hydrograph, interactive_plot
from Weigfun import Weigfun

In [2]:
# This is the correct HBV model and is loaded but hidden on the jupyterbook.

def HBVTeacher(Par,forcing,S_in, hydrograph):
    #HBVpareto Calculates values of 3 objective functions for HBV model

    I_max = Par[0]
    Ce = Par[1]
    Su_max = Par[2]
    beta = Par[3]
    P_max = Par[4]
    T_lag = Par[5]
    Kf = Par[6]
    Ks = Par[7]
    

    Prec = forcing['P'].values
    Qo = forcing['Q'].values
    Etp = forcing['PE'].values


    t_max = len(Prec)
    
    # allocate Si, Su, Sf, Ss, Ei_dt, Ea_dt, Q_tot_dt
    
    Si = np.zeros(t_max)
    Su = np.zeros(t_max)
    Sf = np.zeros(t_max)
    Ss = np.zeros(t_max)
    Ei_dt = np.zeros(t_max)
    Ea_dt =  np.zeros(t_max)
    Q_tot_dt = np.zeros(t_max)
    Qs = np.zeros(t_max)
    Qf = np.zeros(t_max)
    
    # initialize Si, Su, Sf, Ss
    Si[0] = S_in[0]
    Su[0] = S_in[1]
    Sf[0] = S_in[2]
    Ss[0] = S_in[3]

    dt = 1

    #
    # Model 1 SOF1
    for i in range(0, t_max):
        P_dt = Prec[i] * dt
        Ep_dt = Etp[i] * dt
        
        # Interception Reservoir
        if P_dt > 0:
            Si[i] = Si[i] + P_dt 
            Pe_dt = np.maximum(0, (Si[i] - I_max) / dt)
            Si[i] = Si[i] - Pe_dt
            Ei_dt[i] = 0
        else:
        # Evaporation only when there is no rainfall
            Pe_dt = np.maximum(0, (Si[i] - I_max) / dt) #is zero, because of no rainfall
            Ei_dt[i] = np.minimum(Epdt, Si[i] / dt)
            Si[i] = Si[i] - Pe_dt - Ei_dt[i]
        
        if i < t_max-1:
            Si[i+1] = Si[i]
        
        
        # Split Pe into Unsaturated Reservoir and Preferential reservoir
        if Pe_dt > 0:
            Cr = (Su[i] / Su_max) ** beta
            Qiu_dt = (1 - Cr) * Pe_dt # flux from Ir to Ur
            Su[i] = Su[i] + Qiu_dt
            Quf_dt = Cr * Pe_dt #flux from Su to Sf
        else:
            Quf_dt = 0
        
        # Transpiration
        Ep_dt = max(0, Ep_dt - Ei_dt[i])
        Ea_dt[i] = Ep_dt * (Su[i] / (Su_max * Ce))
        Ea_dt[i] = min(Su[i] / dt, Ea_dt[i])
        Su[i] = Su[i] - Ea_dt[i]
        
        # Percolation
        Qus_dt = P_max * (Su[i] / Su_max) * dt # Flux from Su to Ss
        Su[i] = Su[i] - Qus_dt
        
        if i < t_max - 1:
            Su[i+1] = Su[i]
        
        # Fast Reservoir
        Sf[i] = Sf[i] + Quf_dt
        Qf_dt = dt * Kf * Sf[i]
        Sf[i] = Sf[i] - Qf_dt
        if i < t_max-1:
            Sf[i+1] = Sf[i]
        
        # Slow Reservoir
        Ss[i] = Ss[i] + Qus_dt
        Qs_dt = dt * Ks * Ss[i]
        Ss[i] = Ss[i] - Qs_dt
        if i < t_max-1:
            Ss[i+1] = Ss[i]
        
        Q_tot_dt[i] = Qs_dt + Qf_dt
        Qs[i] = Qs_dt 
        Qf[i] = Qf_dt 


    # Check Water Balance
    Sf = Si[-1] + Ss[-1] + Sf[-1] + Su[-1] #final storage
    S_in = sum(S_in) #initial storage
    WB = sum(Prec) - sum(Ei_dt) - sum(Ea_dt) - sum(Q_tot_dt) - Sf + S_in
    # print(WB)
    # Offset Q

    Weigths = Weigfun(T_lag)
    
    Qm = np.convolve(Q_tot_dt, Weigths)
    Qm = Qm[0:t_max]
    forcing['Qm'] = Qm
   
    if hydrograph == 'TRUE':
    ## Plot
    # hour=1:t_max\
        fig, ax = plt.subplots(figsize=(12,8))
        forcing['Q'].plot(label='Obserbed', ax=ax)
        forcing['Qm'].plot(label='Model',  ax=ax)
        ax.legend()
        

    return Qm

    

`````{admonition} Interactive Python Page
:class: tip, dropdown

The code on this page can be used interactively: click {fa}`rocket` --> {guilabel}`Live Code` in the top right corner, then wait until the message {guilabel}`Python interaction ready!` appears.

When this page is activated:
- Several packages will be imported automatically
- Code cells will **not** be executed automatically (you do it!)

````{admonition} Which packages are imported when this page is activated?
:class: note, dropdown
```
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from datetime import date
from plot_hydrograph import plot_hydrograph, interactive_plot
from Weigfun import Weigfun```
````
`````

# Models Exercise 2: Lumped Conceptual Model

In this exercise you will create a lumped conceptual model. In the figure below a schematization of the conceptual model is given.


![image.png](figures/lumped_model.png)

## Part 1

Write down all the different parts of the water balance with the correct units. Make use of the lecture slides and the equations and statements below.   

Water can evaporate from the interception reservoir at the potential rate E$_p$. 

Water can spill from the reservoir when the level S$_i$ reaches I$_{max}$ (mm). 

P$_e$ is partitioned into P$_i$ and P$_f$ according to C$_r$, calculated as: 

$Cr=(S_u/S_{umax})^{\beta}$

Resulting for the fast storage: $Q_{uf} = C_r*P_e$; 

From which it can flow out: $Q_f=K_f*S_f$; 

For the underground storage holds: $Q_{iu} =(1-C_r)*P_e$. 

From here it can evaporate following: $E_a=S_u/(S_{umax}*C_e)*E_p$; 

Or it can percolate further into the ground: $Q_{us} =P_{max}*(S_u/S_{umax})$; 

From this storage it can flow out: $Q_s=K_s*S_s$. 

Q$_f$ and Q$_s$ are summed and routed through the triangular transfer function with base T$_{lag}$

## Part 2 Create HBV model

In this part you will code the lumped model, which is also called the HBV-model. Below the function ```HBVMod``` is given, but as you can see, part of the function is unfinished. Only edit the part below the statement ```UPDATE THE PART BELOW``` and above ```END```. 

After you finish this function, you can plot the hydrograph and see if the function is correct. The plot is interactive, use the sliders to change a parameter and see what their effect is.

In [3]:
def HBVMod(Par, forcing, S_in, hydrograph):
    #HBVpareto Calculates values of 3 objective functions for HBV model

    I_max = Par[0]
    Ce = Par[1]
    Su_max = Par[2]
    beta = Par[3]
    P_max = Par[4]
    T_lag = Par[5]
    Kf = Par[6]
    Ks = Par[7]
    

    Prec = forcing['P'].values
    Qo = forcing['Q'].values
    Etp = forcing['PE'].values


    t_max = len(Prec)
    
    # allocate Si, Su, Sf, Ss, Ei_dt, Ea_dt, Q_tot_dt
    
    Si = np.zeros(t_max)
    Su = np.zeros(t_max)
    Sf = np.zeros(t_max)
    Ss = np.zeros(t_max)
    Ei_dt = np.zeros(t_max)
    Ea_dt =  np.zeros(t_max)
    Q_tot_dt = np.zeros(t_max)
    Qs = np.zeros(t_max)
    Qf = np.zeros(t_max)
    
    # initialize Si, Su, Sf, Ss
    Si[0] = S_in[0]
    Su[0] = S_in[1]
    Sf[0] = S_in[2]
    Ss[0] = S_in[3]

    dt = 1

    #
    # Model 1 SOF1

    ####################################################################
    #                     UPDATE THE PART BELOW                        #  
    ####################################################################

    
    for i in range(0, t_max):
        P_dt = Prec[i] * dt
        Ep_dt = Etp[i] * dt
        
        # Interception Reservoir
        if P_dt > 0:
            Si[i] = Si[i] + P_dt 
            Pe_dt = 
            Si[i] = 
            Ei_dt[i] = 
        else:
        # Evaporation only when there is no rainfall
            Pe_dt = 
            Ei_dt[i] = 
            Si[i] = 
        
        if i < t_max-1:
            Si[i+1] = Si[i]
        
        
        # Split Pe into Unsaturated Reservoir and Preferential reservoir
        if Pe_dt > 0:
            Cr = 
            Qiu_dt = 
            Su[i] = 
            Quf_dt = 
        else:
            Quf_dt = 
        
        # Transpiration
        Ep_dt = 
        Ea_dt[i] =
        Ea_dt[i] = 
        Su[i] =
        
        # Percolation
        Qus_dt = 
        Su[i] = 
        
        if i < t_max - 1:
            Su[i+1] = Su[i]
        
        # Fast Reservoir
        Sf[i] = 
        Qf_dt =
        Sf[i] =
        if i < t_max-1:
            Sf[i+1] = Sf[i]
        
        # Slow Reservoir
        Ss[i] =
        Qs_dt = 
        Ss[i] = 
        if i < t_max-1:
            Ss[i+1] = Ss[i]
        
        Q_tot_dt[i] = 
        Qs[i] = 
        Qf[i] = 

    ####################################################################
    #                             END                                  #  
    ####################################################################
    

    # Check Water Balance
    Sf = Si[-1] + Ss[-1] + Sf[-1] + Su[-1] #final storage
    S_in = sum(S_in) #initial storage
    WB = sum(Prec) - sum(Ei_dt) - sum(Ea_dt) - sum(Q_tot_dt) - Sf + S_in
    print(WB)
    # Offset Q

    Weigths = Weigfun(T_lag)
    
    Qm = np.convolve(Q_tot_dt, Weigths)
    Qm = Qm[0:t_max]
    forcing['Qm'] = Qm
    
    if hydrograph == 'TRUE':
    ## Plot
    # hour=1:t_max\
        fig, ax = plt.subplots(figsize=(12,8))
        forcing['Q'].plot(label='Obserbed', ax=ax)
        forcing['Qm'].plot(label='Model',  ax=ax)
        ax.legend()
        

    return(Qm)


SyntaxError: invalid syntax (2103499071.py, line 56)

Before we can compute the discharge based on your model, we need to import the forcing data from the text file into a dataframe. 

In [4]:
data = pd.read_csv('Forcing.txt', skipinitialspace=True, delimiter='\t', names=['year', 'month', 'day', 'P', 'Q', 'PE'])
forcing = pd.DataFrame()
forcing['P'] = data['P']
forcing['PE'] = data['PE']
forcing['Q'] = data['Q']
forcing.index = data.apply(lambda x: date(int(x.year), int(x.month), int(x.day)), axis=1)
forcing.index = pd.to_datetime(forcing.index, format='%Y-%m-%d')


To create an interactive plot, the sets with minimum and maximum parameters should be given. If you want you can change these values. 

If you run the cell below, an interactive plot will be generated. In this plot you can change the values of the different parameters, play with the values and see their effect on the hydrograph and the NSE value. How many times did you change the parameters to get a reasonable NSE value? 

In [5]:
params = {
    'I_max': {'min': 0, 'max': 10},
    'Ce': {'min': 0.1, 'max': 1},
    'Su_max': {'min': 40, 'max': 800},
    'beta': {'min': 0.5, 'max': 5},
    'P_max': {'min': 0.001, 'max': 0.3},
    'T_lag': {'min': 1, 'max': 10},
    'Kf': {'min': 0.01, 'max': 0.1},
    'Ks': {'min': 0.0001, 'max': 0.01},
}


In [6]:
interactive_plot(HBVMod, forcing, params)

NameError: name 'HBVMod' is not defined

In [7]:
interactive_plot(HBVTeacher, forcing, params)

Output()